In [ ]:
'''
  -----------
    크롤링
  -----------
'''
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from multiprocessing import Process, Queue
from selenium.webdriver.common.keys import Keys
import time
import os


def save_to_excel(rastrant_data:list, local_code:int) :
    
    '''
        코드표
         도쿄 0, 오사카 1, 나고야 2, 요코하마 3, 
         tokyo, Osaka, Nagoya, Yokohama
    
    '''
    # 파일명 뒤에 붙을 이름
    if local_code == 0: local_name = 'tokyo'
    elif local_code == 1: local_name = 'Osaka'
    elif local_code == 2: local_name = 'Nagoya'
    elif local_code == 3: local_name = 'Yokohama'

    # 
    data = [data for rastrant_datam in rastrant_data for data in rastrant_datam]
    df = pd.DataFrame(data)

    if not os.path.exists(f'C:\\Users\\USER\\Desktop\\python\\output_{local_name}.xlsx'):
        with pd.ExcelWriter(f'output_{local_name}.xlsx', mode='w', engine='openpyxl') as writer:
            df.to_excel(writer, index=False)
            print('존재하지않습니다.')
    else:
        with pd.ExcelWriter(f'output_{local_name}.xlsx', mode='a', engine='openpyxl',if_sheet_exists='replace') as writer:
            df.to_excel(writer, index=False)
            print('존재합니다.')

def web_crawling(local_url:str, local_code:int) :
    # 다음 페이지로 이동하는 동작이 3번 안됨 => 다음 페이지가 없음
    # 데이터를 전부 수집했음 -> 종료
    NEXT_PAGE_LIM_CNT = 3

    # 데이터를 추출할 사이트
    URL = 'https://www.tripadvisor.co.kr/Restaurants-g294232-Japan.html'

    # 드라이버 연결하기 (chromedriver.exe 파일을 연결해야한다)
    driver = wd.Chrome('C:\\Users\\USER\\Desktop\\python\\selenium_level4\\chromedriver.exe')
    # get을 사용하여 접속하고 싶은 사이트를 적어준다

    # URL 접속
    driver.get(URL)
    time.sleep(2)

    # https://www.tripadvisor.co.kr/Restaurants-g294232-Japan.html 사이트에 지역들을 클릭한다
    driver.find_element(By.CSS_SELECTOR, local_url).click()
    time.sleep(2)
    
    
    for page_idx in range(1,100) :
        # 현재 페이지에 식당이 없을 때 다음페이지로 이동시켜준다
        # 페이지가 로드되면 바로 실행하고 안되면 10초까지 기다린다
        rastrant_data = list()

        save_idx_2 = page_idx
        if save_idx_2 > 7 : save_idx_2 = 6

        time.sleep(3)
        next_page_move = f'div.deckTools.btm > div > div > a:nth-child({page_idx})'
        # 다음 인덱스로 이동시켜준다
        if page_idx > 1 : driver.find_element(By.CSS_SELECTOR, next_page_move).click()
        # 페이지 없음 카운트 0으로 초기화
        page_none_cnt = 0

        rastrant_idx = 1
        for restaurant_detail_idx in range(rastrant_idx,36) :
            rastrant_code = 1
            print('식당 순위 : ',restaurant_detail_idx, ' 등')
            # 인기있는 식당 페이지 이동( 스폰서 제외 )

            rigion_restaurant = f'div:nth-child({restaurant_detail_idx}) > div > span > div.zqsLh > div.zdCeB.Vt.o > div.tkvCJ.u.F.f.Ff.K > div > span > a'
            

            if page_none_cnt == NEXT_PAGE_LIM_CNT : break

            # 페이지가 로드되기 전까지 기다린다
            # 단 10초가 넘어가면 안된다
            driver.implicitly_wait(10)
            try :
                driver.find_element(By.CSS_SELECTOR, rigion_restaurant).click()
                rastrant_code += 1
            except Exception as e:
                print(e.with_traceback)
                page_none_cnt += 1
                print('식당 없음')
                continue
                
            
            # except가 3회 진행되면 더 이상 페이지가 없는걸로 알고 다음페이지로 이동한다 
            page_none_cnt = 0
            # 탭으로 식당 세부페이지가 뜨기 때문에
            # driver가 가르키는 페이지를 탭으로 표시한다
            # -1은 탭창 중 맨끝에있는 인덱스이다
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(3)

            # 접속한 페이지 html을 가져온다
            rigion_restaurant_html = driver.page_source

            # 뷰티플스푸에 rigion_restaurant_html를 넣어준다
            soup = BeautifulSoup(rigion_restaurant_html,'html.parser')

            # 총 리뷰 밑에 자세하게 나와있는 리뷰 값을 전부 가져온다
            total_rating_detail = soup.select('span.vzATR')
            total_rating_details = list()
            cnt = 0
            try :
                for idx3 in range(0,4) :
                    total_rating_details.append(total_rating_detail[idx3].select('span')[0]['class'][1].replace('bubble_',''))
                    cnt += 1
            except Exception as e:
                # 전체 디테일 리뷰가 없다면 -1을 넣음
                if cnt == 0 : total_rating_details = [-1,-1,-1,-1]
                elif cnt == 3 : total_rating_details.append(-1)
                elif cnt == 2 : total_rating_details.extend([-1,-1])
                elif cnt == 1 : total_rating_details.extend([-1,-1,-1])
                
               
            # 식당 페이지 내에 존재하는 리뷰 페이지를 넘김
            for idx in range(2, 150) :
                save_idx = idx  
                # 페이자 경로 nth-child(1) 값이 7이상이 없음
                if save_idx > 7 : save_idx = 6
                
                try:
                    # 모든 언어로 변경 라디오버튼 클릭
                    driver.find_element(By.CSS_SELECTOR, '.choices.is-shown-at-tablet > .item > .label.container > #filters_detail_language_filterLang_ALL').click()
                except Exception as e:
                    print(e.with_traceback)
                    print('이미 모든언어입니다.')
                time.sleep(1)

                # 현재 페이지 긁음
                soup = BeautifulSoup(driver.page_source,'html.parser')
        
                for idx2 in range(10) :
                    try :
                        # 리뷰가 길면 더보기를 눌러줘야함
                        driver.find_elements(By.CSS_SELECTOR,'.taLnk.ulBlueLinks')[idx2].click()
                        break
                    except Exception as a :
                        print('더 보기가 없습니다')
                time.sleep(2)

                # 더보기 후 한번더 페이지 긁음
                soup = BeautifulSoup(driver.page_source,'html.parser')

                # 리뷰 제목, 코멘트, 별점을 한번에 가져온다
                review_titles = soup.select('.rev_wrap.ui_columns.is-multiline>.ui_column.is-9 > .quote > .title > .noQuotes')
                review_coments = soup.select('.ui_column.is-9 > .prw_rup.prw_reviews_text_summary_hsx > .entry > .partial_entry')
                review_ratings = soup.select('.review-container > .prw_rup.prw_reviews_review_resp > .reviewSelector > .rev_wrap.ui_columns.is-multiline > div.ui_column.is-9')

                # 별점 추출은 따로 빼지 않으면 가끔 오류가 생김
                review_rating = [str(review_rating).split(">")[1].split('"')[1][-2:] for review_rating in review_ratings]
            
                # 필요 데이터를 추출한 후 딕셔너리로 저장
                restaurant_review = [{
                    'code' : f'{local_code}{rastrant_code}',
                    'title':soup.select_one('div.acKDw.w.O > h1').text,
                    'total_rating' : soup.select_one('div:nth-child(2) > span:nth-child(1) > a > svg')['aria-label'].split(' ')[-1].replace('.',''),
                    'total_review_cnt' : soup.select_one('div:nth-child(2) > span:nth-child(1) > a > span').text.split('건')[0].replace(',',''),
                    'total_rating_tasty' : total_rating_details[0],
                    'total_rating_service' : total_rating_details[1],
                    'total_rating_price' : total_rating_details[2],
                    'total_rating_mood' : total_rating_details[3],
                    'address' : soup.select_one('div:nth-child(3) > span:nth-child(1) > span > a').text,
                    'phone' : soup.select_one('span.AYHFM > a')['href'].replace('tel:',''),
                    'price_band' : soup.select_one('div:nth-child(1) > div.SrqKb').text.replace("₩",'').replace(" ",''),
                    'food_type' : soup.select_one('div:nth-child(2) > div.SrqKb').text,
                    'review_title' : review_titles[review_idx].text,
                    'review_coment': review_coments[review_idx].text.replace('\n',' ').replace('더 보기', ''),
                    'review_rating': review_rating                
                } for review_idx in range(0,len(review_ratings))]
                  
                # 딕셔너리를 리스트에 append한다
                rastrant_data.append(restaurant_review)

                # 리뷰 다음페이지 경로
                review_page_idx = f'.pageNumbers > a:nth-child({save_idx})'

                try :
                    # 리뷰 다음페이지 클릭
                    # send_keys키를 사용하여 오류를 방지하였다
                    driver.find_element(By.CSS_SELECTOR, review_page_idx).send_keys(Keys.ENTER)
                except Exception as e :
                    print('리뷰없음')
                    print(e.with_traceback)
                    break

                time.sleep(1)
                print('리뷰 페이지 인덱스 : ', idx, ' 페이지' )
            
            # 탭창 하나를 닫는다
            driver.close()
            time.sleep(1)
            # 첫번째 탭으로 드라이버를 가리킨다
            driver.switch_to.window(driver.window_handles[0])
            # 크롤링한 데이터를 엑셀에 저장한디
            save_to_excel(rastrant_data, local_code)
            time.sleep(1)


if __name__ == "__main__":
    # 첫 화면에서 지역을 선택하는 패스
    regions_url = [f'div:nth-child({hight_idx}) > div:nth-child({width_idx}) > div > div.geo_image > a' for hight_idx in range(1,5) for width_idx in range(1,4)]
    
    '''
        코드표
         도쿄 0, 오사카 1, 나고야 2, 요코하마 3, 
    
    '''
    # 멀티스레드를 이용하여 한번에 여러지역 데이터를 추출
    tokyo = Process(target=web_crawling, args=(regions_url[0],0, ))
    Osaka = Process(target=web_crawling, args=(regions_url[1],1, ))
    #Nagoya = Process(target=web_crawling, args=(regions_url[2],2, ))
    #Yokohama = Process(target=web_crawling, args=(regions_url[3],3, ))

    # 멀티스레드 동작
    tokyo.start()
    Osaka.start()
    #Nagoya.start()
    #Yokohama.start()

In [ ]:
'''
  ---------------------------
    리뷰 제목 및 코멘트 번역
  ---------------------------
'''

import googletrans
import os
import pandas as pd
import time

global writer

# csv 파일 열기
def csv_open(mode) :
    global writer
    if mode == 'a' : writer = open('translate_review_tokyo.csv', 'a', encoding='utf8')
    
# csv 파일 닫기
def csv_close() :
    global writer

    writer.close()

def csv_save(df) :
    global flag, writer

    # 번역 함수를 불러온다
    translator = googletrans.Translator()
    idx_simulated = 1
    idx_actually = idx_simulated
    error_cnt = 0
    csv_open('a')
    for idx in range(idx_simulated,12000) : 
        # 크롤링한 데이터 리뷰제목, 리뷰코멘트를 불러온다 
        title = df.loc[idx_actually,'review_title']
        comment = df.loc[idx_actually,'review_coment']

        try:
            # 제목을 번역한다
            translator_text_title = translator.translate(title,dest='ko').text
            # 코멘트를 번역한다
            translator_text_comment = translator.translate(comment,dest='ko').text
            # 런타임 에러를 대비하여 인덱스를 두개 만들었다
            idx_actually += 1
            error_cnt = 0
        except Exception as e:
            # 에러카운트가 3이 되면 idx_actually 인덱스를 하나 올려준다
            if error_cnt == 3: idx_actually += 1
            error_cnt += 1
            print(e.with_traceback)
            continue
            
        # 번역한 데이터를 데이터프레임으로 변경해준다
        df2 = pd.DataFrame([{
            'title' : translator_text_title.strip(),
            'comment' : translator_text_comment.strip()
        }], index=[idx])

        # 현재 진행상황을 볼 수 있다
        print(df2)

        # csv를 오픈하여 데이터를 저장하고 닫아준다
        csv_open('a')
        df2.to_csv(writer, index=True)
        csv_close()

# 엑셀을 불러오고, 함수를 호출한다
df = pd.read_excel(io='C:\\Users\\USER\\Desktop\\python\\output_tokyo.xlsx')
csv_save(df)


In [ ]:
'''
  ---------------------------
    csv 번역본 데이터 정제
  ---------------------------
'''
import pandas as pd
import os

# 파일 저장 함수
def save_exc(df) :
    # 경로상에 존재 유무 확인
    if not os.path.exists(f'C:\\Users\\USER\\Desktop\\python\\review[title,comment].xlsx'):
        # 없다면 파일을 새로만들어 데이터를 입력한다
        with pd.ExcelWriter(f'review[title,comment].xlsx', mode='w', engine='openpyxl') as writer:
            df.to_excel(writer, index=False)
            print('존재하지않습니다.')
    else:
       # 있다면 기존파일에 데이터를 추가한다
        with pd.ExcelWrit
        er(f'review[title,comment].xlsx.xlsx', mode='a', engine='openpyxl',if_sheet_exists='replace') as writer:
            df.to_excel(writer, index=False)
            print('존재합니다.')

# 암시적 리스트를 생성해준다
review_data = list()
# csv 번역본을 불러온다
with open(r'C:\\Users\\USER\\Desktop\\python\\translate_review_tokyo.csv', 'r', encoding='utf-8') as review :
    for review2 in review.readlines():
        # 띄어쓰기, 필요없는 데이터 등을 정제한다
        review2 = review2.strip()
        if review2 == "" : continue
        if review2 == ',title,comment' : continue
        detail = review2.split(',')
        comment = ''
        # 쉼표로 구분되기 때문에 코멘트는 많은 배열로 스플릿된다
        # => 두번째 배열 뒤로는 코멘트이기 때문에 전부 조인해준다
        if len(detail) > 2 : comment = ''.join(detail[2:])
        else : comment = detail[2]
        
        # 리스트에 append한다
        review_data.append({
            'idx' : detail[0],
            'title' : detail[1].replace('"',''),
            'comment' : comment
        })
        print(detail[0])

# 데이터를 불러오고 함수를 호출한다
rastrant_review = pd.DataFrame(review_data)
save_exc(rastrant_review)
    

In [ ]:
'''
  ----------------------------------------------------------------------
    데이터 시각화를 위한 준비 (불용어 추가 후 필요한 키워드를 가져온다)
  ----------------------------------------------------------------------
'''

from wordcloud import WordCloud, STOPWORDS
import pandas as pd
from konlpy.tag import Mecab

# 암시적 리스트 만들기
stopword_text = list()

# txt파일로 정리한 불용어를 불러 리스트에 저장한다
with open('/content/drive/MyDrive/팀별 활동/팀플자료/불용어 정리.txt','r',encoding='utf-8') as reader :
  stopword_text = str(reader.read()).split(',')

stopwords = set(STOPWORDS)
# 리뷰 코멘트를 전부 합쳐준다
review_comments = ''.join(map(str,review.review_coment))
# stopword_text에 저장된 불용어들을 적용한다
for value in stopword_text :
  value = value.replace("'",'')
  stopwords.add(value)

wc2 = WordCloud(
    font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # 폰트지정
    max_words = 300, # 빈도 서열 300위까지 클라우드에 표현
    stopwords = stopwords,
    background_color='white' # 마스크 베이스를 흰색 세팅
)
wc_gen = wc2.generate(review_comments)
stopwords_key = [k for k,_ in wc_gen.words_.items()]
print(stopwords_key)



In [ ]:
import numpy as np
import pandas as pd

# 최종적으로 정리된 파일을 가져온다
review = pd.read_excel('/content/drive/MyDrive/팀별 활동/팀플자료/review_tokyo_final.xlsx')
# 테그 정리
tag_list = ['quality_tag','fresh_tag', 'mood_comfortable_tag', 'mood_happy_tag', 'mood_etc_tag', 'service_tag', 'language_tag','clean_tag', 'cost_tag', 'location_tag', 'bad_etc_tag', 'bad_cost_tag','bad_service_tag', 'bad_quality_tag', 'bad_clean_tag', 'wait_tag']

# 식당 이름
rastaurant_list = ['Downtown B’s Indian Kitchen',
  'Elio Locanda Italiana',
  'Gyopao Gyoza Roppongi',
  'Gyopao Gyoza Shinjuku',
  'Gyuan',
  'Gyukatsu Ichi Ni San',
  'Gyukatsu Motomura Shibuya',
  'Heart of Hearts',
  'Ikuta',
  'Imahan Honten',
  'Kiji, Marunouchi',
  'Kobe Beef 511',
  'Kushiyaki Bistro Fukumimi Shinjuku',
  'Mutekiya',
  'NINJA Café & Bar',
  'Nabezo Ikebukuro Sunshine 60th Street',
  'New York Grill',
  'Nihonshu Genka Sakagura Shinjuku East Entrance',
  'Rokkasen',
  'Sky Restaurant 634 (Musashi)',
  'Sushizanmai Tsukijiekimae-Ten',
  'Umegaoka Sushi No Midori Sohonten Shibuya',
  'Umegaoka Sushino Midori Ginza',
  'star star',
  '노부 도쿄',
  '이타마에 스시\u3000아카사카',
  '큐베이']


# 아래 용어가 포함되면 태그컬럼 1을 올려줌
quality_tag = ['풍부한 향', '풍부한 맛','다양한',  '품질이 높', '고품질', '육즙', '풍미', '부드러', '부드럽', '육즙이','고품질','우수한 가지', '우수한 조개', '우수한 스시', '우수한 된장', '우수한 고기', '우수한 원료', '우수한 맥주', '우수한 버거', '우수한 초밥', '우수한 품질', '우수한 사시미', '우수한 국물', '우수한 국수', '우수한 음식']
quality_tag_except = {'다양한':'할인','다양한':'음악','다양한':'관점','다양한':'상세한 주문','다양한':'고유','다양한':'쿠폰','다양한':'장면','다양한':'사람','다양한':'이기심','다양한':'유리한','부드러':'운 분위기','부드러':'운 조명','부드러':'운 치유','부드러':'운 즐거운'}

fresh_tag =['신선', '싱싱','탁월한 신선도']
fresh_tag_except = {'신선' : '한공기', '신선' : '한지 아닌지', '신선' : '하지 않', '신선' : '하지도 않'}

mood_comfortable_tag = [ '편안', '아늑', '편안']
mood_comfortable_tag_except = {'편안' : '하게 만들지 않', '편안' : '한 분위기가 아니', '편안' : '하지 않', '편안' : '덜 편안한' }

mood_happy_tag  = [ '재미', '재밌', '즐기', '즐거', '웃음']
mood_happy_tag_except = {'즐기':'지 않','즐기':'려면'}

mood_etc_tag = ['전통', '환상적', '환상이', '독특', '데이트', '고급스럽', '우수한 분위기' ]
mood_etc_tag_except = {'전통' : '적인 위치', '환상적': '인 것은 없', '환상적' : '이지 않', '환상적' : '인 위치', '독특' : '한 할인',  '독특':'한 트위스트', '독특':'한 게임', '데이트' : '업'}

service_tag = ['친절', '친화적', '예의바르', '협조적', '미소', '세심', '친밀', '정중', '우수한 서비스', '탁월한 서비스', '훌륭한 서비스', '좋은 서비스', '완벽한 서비스', '빠른 서비스', '서비스 경험은 훌륭', '서비스 태도는 훌륭', '서비스 아주 좋', '서비스 직원들은 훌', '놀라운 서비스', '전문적인 서비스', '서비스가 양호', '서비스 매우 세심', '서비스는 완벽', '서비스도 완벽', '서비스에 만', '서비스는 환', '서비스는 놀랍', '서비스는 편안', '서비스는 좋았', '서비스는 최고']
service_tag_except = tag_exept = {'친절' : '하지 않', '친화적': '이지 않', '미소' : '가 거의', '미소' : '는 전혀', '미소': ' 된장국', '미소' : '시루', '미소':' 수프', '미소' : '국',  '미소': ' 샐러드', '미소' : '구운', '세심':'한 균형', '세심':'하지 않', '친밀':'한 식당이 아', '친밀':'하지 않', '친밀':'한 위치', '좋은 서비스':'와 화', '좋은 서비스':'는 아니' }

language_tag  = [ '다국어', '영어', '여러 언어', '다양한 언어', '유창하게', '유창한']
language_tag_except = {'영어':'를 잘 하지','영어':'를 많이하지','영어':'를 구사하지'}

clean_tag = ['깨끗', '깔끔', '청결', '위생적']
clean_tag_except = {'깨끗':'하지'}

cost_tag = ['저렴', '좋은 가격', '가성비', '싸고', '합리적', '할인', '무제한', '비싸지는 않', '비싸지 않', '세트', '우수한 가격']
cost_tag_except = { '저렴':'하진 않', '저렴':'하지 않', '저렴':'하지는 않', '저렴':'한 가격이 아니', '저렴':'한 식사는 아니', '저렴':'합니까?', '저렴':'곳이 아닌', '가성비': '가 좋다는데', '싸고':'비싸고', '싸고':'감싸고', '싸고':'둘러싸고', '합리적':'인 영어', '합리적':'입니까?', '합리적':'인 예산', '합리적':'이지는 않', '합리적':'인 대기', '합리적':'인 크기', '합리적':'으로 영어','합리적':'채소', '합리적':'인 취향', '합리적':'으로 비쌉', '합리적':'으로 좋은 영어', '합리적':'인 환경', '세트': '에 앉았', '세트':'영화 세트', '세트':'를 기다리' }

location_tag = [ '쇼핑몰에서 쉽게 찾', '매우 쉽게 찾', '중심가', '중심에 있', '찾기가 쉽습', '찾기가 편리', '찾기가 매우 쉽', '찾기가 매우 쉬', '찾기가 아주 쉽' ]
location_tag_except = {'' : ''}

bad_etc_tag =[ '실망', '최악','미치지 못', '만족스럽지 않', '최악의 초밥', '가게가 최악','최악의 요리', '최악의 스테이크', '최악의 상황은 서비스', '최악의 식사', '최악의 가치', '최악의 얼','최악의 쇠고기', '품질 최악', '쌀은 최악', '음식은 끔찍', '맛이 끔찍', '애피타이저는 끔찍', '끔찍한 느낌','끔찍한 식사', '케이크는 끔찍', '끔찍한 고통','현금을 날려']
bad_etc_tag_except = {'최악':'의 법안','실망':'시키지 않', '실망':'과 같지 않','실망':'하지 않','실망':'실망할 일은 없' ,'실망':'실망하지만','나쁜':'나쁜 표현', '나쁜': '점에서','나쁜':'별로 나쁜' ,'나쁜':'싶지 나쁜' ,'나쁜':'서비스', '나쁜':'날씨', '나쁜':'균형', '나쁜':'재즈', '흔한':'서비스', '미치지 못':'영향을 미치지 못'}

bad_cost_tag =  ['비싼','비싸','최악의 법안은 청구서']
bad_cost_tag_except = {'':''}

bad_service_tag = ['서비스는 끔찍','끔찍한 서비스','서비스가 느린', '서비스가 매우 느리', '서비스가 확 떨어','인종차별' '와사비 테러','불친절''무례','무시','소홀', '인성도 최악', '서비스는 최악', '무관심 서비스', '태도는 너무 끔찍','나쁜 표현']
bad_service_tag_except = {'':''}

bad_quality_tag = [ '짠', '짜고', '비린', '비렸', '비립', '신선하지 않','맛이 좋지 않','맛없', '기름진']
bad_quality_tag_except = {'':''}

bad_clean_tag = ['바퀴벌레의', '지저분한 외관', '더럽고', '에서 담배','좌석에서 담배' ,'흡연','담배 초밥', '담배 연기','담배를 맹', '담배 피는', '담배를 피우는 것은 매우 나쁩']
bad_clean_tag_except = {'':''}

wait_tag = ['항상 줄','긴 줄','웨이팅','분 동안 기다','분을 기다','줄을 서서 기다', '줄을 기','대기줄','2 시간을 기다', '대기열이 있', '쯤 기다','대기표','간이 걸','오래 기다려','2 시간을 기다', '1 시간을 기다', '두 시간을 기다', '긴 시간을 기다','2시간을 기다','오랜 시간을 기다', '반 시간을 기다','분 이상 기다','분 이상을 기다', '1시간을 기다','5 시간을 기다','3 시간을 기다', '상당한 시간을 기다' ,'3시간을 기다', '한 시간을 기다', '한시간을 기다', '몇 시간을 기다','이상 줄','이상 기다려야', '시간 정도 기다','시간의 대기열', '약간의 대기열', '뒤에 대기열', '대기열은 매우 길', '대기열이 너무 많', '분 대기열', '항상 대기열', '대기열을 준비']
wait_tag_except = {'':''}

tags = [quality_tag, fresh_tag, mood_comfortable_tag, mood_happy_tag, mood_etc_tag, service_tag, language_tag, clean_tag, cost_tag, location_tag, bad_etc_tag, bad_cost_tag, bad_service_tag, bad_quality_tag, bad_clean_tag, wait_tag]
tags_except = [quality_tag_except, fresh_tag_except,mood_comfortable_tag_except ,mood_happy_tag_except, mood_etc_tag_except, service_tag_except,language_tag_except,clean_tag_except,cost_tag_except,location_tag_except,bad_etc_tag_except,bad_cost_tag_except,bad_service_tag_except,bad_quality_tag_except,bad_clean_tag_except, wait_tag_except]

# 데이터 프레임에 새로운 태그 컬럼을 만들어주는 함수
def make_tag_colums(review) :
  global tag_list

  for tag in tag_list:
    review[tag] = 0

  return review


# 아래 내용이 포함돼 있으면 각각의 태그에 1을 부여한다
# 리뷰하나당 연산한다
def tag_review_check(idx:int, review) :
  global tag_list, tags_except, tags

  flag = False
  for i,tag in enumerate(tag_list) :
    restaurant_status_tag = tags[i]
    status_tag_exept = tags_except[i]
    
    for status in restaurant_status_tag :
      try:
        # 리스트에 단어 있는지 비교
        review_comment = review['review_coment'][idx]
        text_check = review_comment.find(status)

        if text_check == -1 : continue
      except Exception as e :
        # 빈 코멘트일 경우 0을 부여 후 리턴
        review[tag][idx] = 0
        break

      # 제외 단어 ex) '풍미'라는 긍정단어 찾을 시 -> 풍미가 없다(제외) 
      for key,value in status_tag_exept.items() :
        # 제외단어와 포함시킬 단어가 똑같다면
        # 포함시킬 단어에서부터 위치부터 제외단어가 포함되는지 확인한다
        if key == status :
          except_check = review_comment[text_check - 5 : text_check + len(value) + 1].find((key+value))

          if except_check > -1 : 
            flag = True
            break

      if flag : continue
      
        # 포함이면 1을 부여한다음 브레이크 해준다
      review[tag][idx] = 1
      break

def total_sum(review) :
  global rastaurant_list

  # 식당별 태그 개수를 정리하는 리스트  
  total_score = list()

  # 식당 이름이 없을때까지 동작
  for rastaurant in rastaurant_list:
    tag_sum = list()
    tag_sum.append(rastaurant)

    # 태그가 없을 때까지 반복한다
    for idx,tag in enumerate(tag_list):
      cnt = 0

      # 같은 식당이름에 데이터프레임을 불러온다
      # 같은 식당이 아닐때까지 반복한다
      for review_tag_cnt in review[review['title'] == rastaurant][tag] :
        
        # 같은 태그 합을 구한다
        cnt += review_tag_cnt
      tag_sum.append(cnt)
    total_score.append({
        'title'           :tag_sum[0],
        'quality'         :tag_sum[1],
        'fresh'           :tag_sum[2],
        'mood_comfortable':tag_sum[3],
        'mood_happy'      :tag_sum[4],
        'mood_etc'        :tag_sum[5],
        'service'         :tag_sum[6],
        'language'        :tag_sum[7],
        'clean'           :tag_sum[8],
        'cost'            :tag_sum[9],
        'location'        :tag_sum[10],
        'bad_etc'         :tag_sum[11],
        'bad_cost'        :tag_sum[12],
        'bad_service_tag' :tag_sum[13],
        'bad_quality_tag' :tag_sum[14],
        'bad_clean_tag'   :tag_sum[15],
        'waiting_tag'     :tag_sum[16],
        'quality_score'   :tag_sum[1] - tag_sum[14],
        'cost_score'      :tag_sum[9] - tag_sum[12],
        'service_score'   :tag_sum[6] - tag_sum[13],
        'languge_score'   :tag_sum[7] * 0.5,
        'clean_score'     :tag_sum[8] - tag_sum[15],
        'total_score'     :(tag_sum[1] - tag_sum[14]) + tag_sum[2] + (tag_sum[9] - tag_sum[12]) + (tag_sum[6] - tag_sum[13]) + (tag_sum[7] * 0.5) + (tag_sum[8] - tag_sum[15]) + (tag_sum[3] + tag_sum[4] + tag_sum[5]) + tag_sum[10] - tag_sum[11] - tag_sum[16]
    })
  return total_score

# excel 저장 함수
def save_exc(df) :
    with pd.ExcelWriter('/content/drive/MyDrive/팀별 활동/데이터점수5.xlsx', mode='w', engine='openpyxl') as writer:
        df.to_excel(writer, index=False)
        print('존재하지않습니다.')


# 메인
review = make_tag_colums(review)
MAX_REVIEW = 11405
for i in range(MAX_REVIEW):
  tag_review_check(i, review)


total_score = total_sum(review)
df = pd.DataFrame(total_score)


save_exc(df)

FileNotFoundError: ignored

In [ ]:
'''
  -------------------------------------
              시각화
  -------------------------------------
'''

# 한글 폰트 설치
#!sudo apt-get install -y fonts-nanum

# 폰트를 matplotlib 적용한다
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf # 만약 런타입 재가동 버튼이 발생하면 재가동 한다
# 위는 폰트 다운받는 코드
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rc('font',family='NanumBarunGothic')
import pandas as pd
from pandas import Series

df=pd.read_excel('/content/drive/MyDrive/팀별 활동/데이터점수4-3.xlsx')
data_list=[285,299,1410,387,453,115,233,101,152,158,245,542,144,623,115,110,896,201,91,346,919,1095,577,149,349,509,900]  #가게별 리뷰 총개수를 받아온 리스트
count_data=Series(data=data_list) #시리즈화 시켰다(기존 df와 순서 맞춰서 넣었다)
df['review_total_cnt']=count_data # 컬럼화 

df['quality/review_total_cnt']=df.quality/df.review_total_cnt
df['fresh/review_total_cnt']=df.fresh/df.review_total_cnt
df['mood_comfortable/review_total_cnt']=df.mood_comfortable/df.review_total_cnt
df['mood_happy/review_total_cnt']=df.mood_happy/df.review_total_cnt
df['mood_etc/review_total_cnt']=df.mood_etc/df.review_total_cnt
df['service/review_total_cnt']=df.service/df.review_total_cnt
df['language/review_total_cnt']=df.language/df.review_total_cnt
df['clean/review_total_cnt']=df.clean/df.review_total_cnt
df['cost/review_total_cnt']=df.cost/df.review_total_cnt
df['location/review_total_cnt']=df.location/df.review_total_cnt
df['waiting_tag/review_total_cnt']=df.waiting_tag/df.review_total_cnt       # 그래프를 만들기 위해 비율용 컬럼들 생성


for keyword in df.columns[-11:]:
  df = df.sort_values(by=[keyword], ascending=True)
  df.plot.barh(x='title',y=keyword,rot=0,figsize=(5,10))  # 태그별  가게들의 태그/리뷰수비율을 보여주는  그래프 

df.plot.barh(x='title',y='store_score',rot=0,figsize=(5,10))   # 이 그래프가 최종 점수 그래프


plt.figure(figsize=(20,5))
plt.plot(df.title,df.quality_score,label='quality')
plt.plot(df.title,df.service,label='service')
plt.plot(df.title,df.cost_score,label='cost')
plt.plot(df.title,df.languge_score,label='language')
plt.plot(df.title,df.clean,label='clean')
plt.xticks(rotation=90)
plt.legend()
plt.show()                     # 스코어컬럼들을 모아둔 선형(liner)그래프

df.drop(['quality_score', 'cost_score',
       'service_score', 'languge_score', 'clean_score', 'total_score',
       'total_keyword_cnt', 'store_score',
       'review_total_cnt', 'quality/review_total_cnt',
       'fresh/review_total_cnt', 'mood_comfortable/review_total_cnt',
       'mood_happy/review_total_cnt', 'mood_etc/review_total_cnt',
       'service/review_total_cnt', 'language/review_total_cnt',
       'clean/review_total_cnt', 'cost/review_total_cnt',
       'location/review_total_cnt', 'waiting_tag/review_total_cnt'],axis=1,inplace = True)  #사용했던 비율용 칼럼들과 스코어컬럼들을 삭제했다


df2 = pd.pivot_table(df,index='title').T #파이 그래프로 표현하기 유용하게 피벗과 T(축바꾸기)사용
for col in df2.columns:
  df2[col].plot( kind='pie',fontsize=10,  ylabel  ='',rot =0,figsize=(10,10) )
  plt.title(col);
  plt.show()  #가게별 태그비율 파이 그래프